In [3]:
import tensorflow as tf
import numpy as np 
import pandas as pd 
from tensorflow.keras.preprocessing import image_dataset_from_directory                                                          # type: ignore
from tensorflow.keras.layers import Dense,Dropout,Conv2D,MaxPooling2D,Flatten,Rescaling,RandomFlip,RandomZoom,RandomRotation     # type: ignore
from tensorflow.keras.models import Sequential                                                                                   # type: ignore
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint                                                             # type: ignore
import matplotlib.pyplot as plt      
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix                                                                                            
import os
import sys
import keras
import sklearn
print("TensorFlow version:", tf.__version__)
print("Keras version:", keras.__version__)
print("Python version:", sys.version)
print("NumPy version:", np.__version__)
print("Pandas version:", pd.__version__)
print("Scikit-learn version:", sklearn.__version__)
print("OS:", os.name)
print("CPU Count:", os.cpu_count())

TensorFlow version: 2.20.0
Keras version: 3.11.3
Python version: 3.11.8 (tags/v3.11.8:db85d51, Feb  6 2024, 22:03:32) [MSC v.1937 64 bit (AMD64)]
NumPy version: 2.2.6
Pandas version: 2.3.3
Scikit-learn version: 1.7.2
OS: nt
CPU Count: 8
